# <font color="#418FDE" size="6.5" uppercase>**Project Design**</font>

>Last update: 20260130.
    
By the end of this Lecture, you will be able to:
- Define a concrete project goal, dataset choice, and evaluation metrics for a PyTorch capstone. 
- Select an appropriate model architecture and training strategy aligned with project constraints. 
- Plan an experiment schedule that includes baselines, ablations, and potential optimizations. 


## **1. Scoping PyTorch Projects**

### **1.1. Selecting a Project Domain**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_10/Lecture_A/image_01_01.jpg?v=1769783154" width="250">



>* Domain choice shapes data, models, evaluation
>* Consider users, their decisions, and potential risks

>* Start from your interests, then check feasibility
>* Choose a focused domain with enough usable data

>* Consider ethical, social, and data sensitivity issues
>* Balance ambition with fairness, risk, and practicality



### **1.2. Checking Data Sources**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_10/Lecture_A/image_01_02.jpg?v=1769783170" width="250">



>* Check if data truly matches project goal
>* Ensure data is recent, representative, and aligned

>* Check dataset quality, balance, and missing information
>* Data issues shape goals, metrics, and preprocessing

>* Check licenses, privacy rules, and data access
>* Ensure projects stay ethical, compliant, and shareable



### **1.3. Success Criteria Design**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_10/Lecture_A/image_01_03.jpg?v=1769783189" width="250">



>* Define measurable, realistic metrics before training models
>* Align metrics with real-world costs and priorities

>* Combine performance metrics with real-world constraints
>* Set explicit targets for accuracy, speed, and robustness

>* Compare models against simple, well-chosen baselines
>* Define clear improvement margins and stopping conditions



## **2. Choosing Model Architectures**

### **2.1. Task Aligned Architectures**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_10/Lecture_A/image_02_01.jpg?v=1769783215" width="250">



>* Match architecture to the task’s prediction type
>* Use data structure to guide model family choice

>* Match architectures to locality, hierarchy, connectivity patterns
>* Use domain structures to learn meaningful representations

>* Match architectures to outputs and project constraints
>* Balance accuracy, calibration, latency, robustness, and updates



### **2.2. Reuse or Build Anew**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_10/Lecture_A/image_02_02.jpg?v=1769783239" width="250">



>* Choose reuse or new model by constraints
>* Reuse for small data; build for unique, large

>* Match pretrained models to your data domain
>* Choose model family, size, and adaptation strategy

>* Design new models when reuse conflicts constraints
>* Weigh flexibility, resources, deployment, and performance tradeoffs



### **2.3. Scaling to Hardware Limits**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_10/Lecture_A/image_02_03.jpg?v=1769783262" width="250">



>* Balance model size with hardware and time
>* Prefer smaller models that allow rapid experimentation

>* Start small, then gradually scale model complexity
>* Increase size until hardware or time constraints appear

>* Use configs that ease memory or time limits
>* Align model and training plan with resources



In [ ]:
#@title Python Code - Scaling to Hardware Limits

# This script shows scaling models to hardware limits.
# We simulate small and larger models with tiny data.
# Focus on batch size memory and training time tradeoffs.

# !pip install tensorflow==2.20.0.

# Import required standard libraries.
import os
import time
import random

# Import numpy for numeric operations.
import numpy as np

# Import tensorflow and check version.
import tensorflow as tf

# Set deterministic seeds for reproducibility.
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Print tensorflow version in one short line.
print("TensorFlow version:", tf.__version__)

# Detect available device type for information.
physical_gpus = tf.config.list_physical_devices("GPU")
if physical_gpus:
    device_type = "GPU"
else:
    device_type = "CPU"

# Print detected device type briefly.
print("Running on device type:", device_type)

# Create a tiny synthetic image classification dataset.
num_samples = 512
img_height = 28
img_width = 28
num_classes = 10

# Generate random images with small values.
images = np.random.rand(num_samples, img_height, img_width, 1).astype("float32")

# Generate random integer labels for classes.
labels = np.random.randint(0, num_classes, size=(num_samples,))

# Convert labels to categorical one hot encoding.
labels_categorical = tf.keras.utils.to_categorical(labels, num_classes)

# Validate shapes before building datasets.
assert images.shape[0] == labels_categorical.shape[0]

# Define a helper function to build datasets.
def make_dataset(batch_size):
    ds = tf.data.Dataset.from_tensor_slices((images, labels_categorical))
    ds = ds.shuffle(buffer_size=num_samples, seed=seed_value)
    ds = ds.batch(batch_size)
    return ds

# Define a small baseline convolutional model.
def build_small_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(8, (3, 3), activation="relu", input_shape=(img_height, img_width, 1)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(num_classes, activation="softmax"),
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

# Define a larger model that stresses memory more.
def build_large_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(img_height, img_width, 1)),
        tf.keras.layers.Conv2D(32, (3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(num_classes, activation="softmax"),
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

# Prepare two datasets with different batch sizes.
small_batch_size = 32
large_batch_size = 128
train_ds_small = make_dataset(small_batch_size)
train_ds_large = make_dataset(large_batch_size)

# Function to train a model and measure time.
def timed_train(model, dataset, epochs, description):
    start = time.time()
    history = model.fit(dataset, epochs=epochs, verbose=0)
    end = time.time()
    duration = end - start
    final_acc = history.history["accuracy"][-1]
    print(description, "time_sec=", round(duration, 3), "acc=", round(float(final_acc), 3))

# Train small model with comfortable batch size.
small_model = build_small_model()
timed_train(small_model, train_ds_small, epochs=3, description="Small model, batch 32:")

# Train large model with same batch size.
large_model = build_large_model()
timed_train(large_model, train_ds_small, epochs=3, description="Large model, batch 32:")

# Train large model with larger batch size.
large_model_big_batch = build_large_model()
timed_train(large_model_big_batch, train_ds_large, epochs=3, description="Large model, batch 128:")

# Show how many batches each dataset uses.
num_batches_small = len(list(train_ds_small))
num_batches_large = len(list(train_ds_large))
print("Batches with batch 32:", num_batches_small)
print("Batches with batch 128:", num_batches_large)

# Final line prints simple summary message.
print("Example complete: compare speed versus model and batch choices.")




## **3. Designing Experiment Plans**

### **3.1. Defining Strong Baselines**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_10/Lecture_A/image_03_01.jpg?v=1769783367" width="250">



>* Baselines are simple methods setting minimum performance
>* They mirror realistic non-deep solutions for comparison

>* Choose simple, realistic models as careful baselines
>* Document preprocessing, training, and evaluation for fairness

>* Use multiple baselines from simple to advanced
>* Compare gains to justify complexity and deployment costs



### **3.2. Targeted Ablation Strategies**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_10/Lecture_A/image_03_02.jpg?v=1769783386" width="250">



>* Use controlled ablations to test component importance
>* Plan focused ablations ahead to structure experiments

>* Ablate uncertain, novel, or costly components first
>* Compare variants to measure each design choice’s value

>* Prioritize informative ablations within time, compute limits
>* Predefine training, metrics, thresholds, and document rationale



### **3.3. Optimization roadmap**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_10/Lecture_A/image_03_03.jpg?v=1769783405" width="250">



>* Turn vague tuning into a staged, structured plan
>* Start with cheap tweaks, then advanced techniques deliberately

>* Plan phased experiments with clear themes, budgets
>* Progress from stability to capacity, data, advanced techniques

>* Set clear improvement thresholds and stopping rules
>* Preplan reactions to outcomes for disciplined experimentation



# <font color="#418FDE" size="6.5" uppercase>**Project Design**</font>


In this lecture, you learned to:
- Define a concrete project goal, dataset choice, and evaluation metrics for a PyTorch capstone. 
- Select an appropriate model architecture and training strategy aligned with project constraints. 
- Plan an experiment schedule that includes baselines, ablations, and potential optimizations. 

<font color='yellow'>Congratulations on completing this course!</font>